## Eager Execution (Default in TF 2.0) vs. Graph Execution & Session in TensorFlow

The words “Tensor” and “Flow” mean that tensors (data) flow through the computational __graph__. (Default in TF1.0)

What are __graphs__?

In the previous codes, you ran TensorFlow eagerly. This means interactively TensorFlow operations are executed by Python, operation by operation, and returning results back to Python.

There are no graphs in Eager execution, we can’t have the graph magic (automatic differentiation). We have to rely on tf.GradientTape to record operations.


In [1]:
import tensorflow as tf
tf.executing_eagerly() # Default mode

True

In [2]:
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

### __The speed comparison of Eager vs Graph model__

In [3]:
# TensorFlow imports
import timeit
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Flatten, Dense

# Model building
inputs = Input(shape=(28, 28)) 
x = Flatten()(inputs) 
x = Dense(256, "relu")(x)
x = Dense(256, "relu")(x) 
x = Dense(256, "relu")(x) 
outputs = Dense(10, "softmax")(x) 

input_data = tf.random.uniform([100, 28, 28])

# Eager Execution
eager_model = Model(inputs=inputs, outputs=outputs)
print("Eager time:", timeit.timeit(lambda: eager_model(input_data), number=1000))

#Graph Execution 
graph_model = tf.function(eager_model) # Wrap the model with tf.function 
print("Graph time:", timeit.timeit(lambda: graph_model(input_data), number=1000))

2022-09-21 14:24:50.388926: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 171 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0004:04:00.0, compute capability: 7.0


Eager time: 1.924209255957976
Graph time: 0.7454432990052737


Practically, when we use high-level API like Keras, The Keras mode is about defining the __graph__, __compile__, and then __run__ it in GPU later. However, if we need to build new functions and want to execute each step interactively, then we need to annotate functions with the @tf.function decorator. By annotating your functions you tell TensorFlow to run those operation in a optimized graph in the GPU. This generated graph will be run very efficiently into the GPU.

A good reference if you are interested to learn more: https://towardsdatascience.com/tensorflow-2-1-a-how-to-a3e6400d0899